In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import descartes
import geopandas as gpd
import numpy as np
from shapely.geometry import Point, Polygon
import folium
import branca

In [8]:
community_map = gpd.read_file('SHP/SF_SHP/Original_Geojson/neighbourhoods.geojson')
community_map = community_map.rename(columns={'neighbourhood':'community'})

df = pd.read_csv('./Data/SF_Crimes.csv')
#filter for crimes committed in 2021 or 2022
df = df.loc[df['Incident Year'] > 2020]
crs = {'init' : 'epsg:4326'}
df = df.dropna(subset=['Latitude'])
df= df[df['Latitude'] != 0]

geometry = [Point(xy) for xy in zip(df["Longitude"], df["Latitude"])]
geometry[:3]


geo_df = gpd.GeoDataFrame(df, crs = crs, geometry = geometry)

/opt/anaconda3/envs/geo_env/lib/python3.10/site-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [9]:
df.to_csv('./Data/SF_Crimes.csv', index=False)

In [3]:
# community_map['community'].unique()

In [11]:
# taken from https://towardsdatascience.com/tagging-a-location-to-a-shapefile-area-using-geopandas-5d74336128bf
geo_df['community'] = ''
for idx in range(community_map.shape[0]):
    #For every lat/long, find if they reside within a community
    pip = geo_df.within(community_map.loc[idx, 'geometry'])
    if pip.sum() > 0:
        geo_df.loc[pip, 'community']  = community_map.loc[idx, 'community']

In [12]:
# taking group by and then reverting back to data frame to use
# https://technology.amis.nl/data-analytics/convert-groupby-result-on-pandas-data-frame-into-a-data-frame-using-to_frame/
crimebycommunity = geo_df.groupby(['community'])['community'].count().to_frame(name = 'count').reset_index()


crimebycommunity['community'].replace('', np.nan, inplace=True)
crimebycommunity = crimebycommunity.dropna(subset=['community'])


map_and_stats = community_map.merge(crimebycommunity, on="community")

map_and_stats_2 = map_and_stats[['community','geometry','count']]

In [13]:
map_and_stats_2.to_file("SHP/SF_SHP/Summarized_Shp/SFCrime.shp")

/opt/anaconda3/envs/geo_env/lib/python3.10/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [14]:
map_and_stats_2 = gpd.read_file("SHP/SF_SHP/Summarized_Shp/SFCrime.shp")

In [15]:
# https://vverde.github.io/blob/interactivechoropleth.html

x_map = map_and_stats_2.centroid.x.mean()
y_map = map_and_stats_2.centroid.y.mean()



mymap = folium.Map(location=[y_map, x_map], zoom_start=12,tiles=None)
folium.TileLayer('CartoDB positron',name="Light Map",control=False).add_to(mymap)


myscale = (map_and_stats_2['count'].quantile((0,0.25,0.9,1))).tolist()

/var/folders/9n/1h5nn1bn3ns9xx1s5nvxy5n40000gn/T/ipykernel_61447/136192827.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  x_map = map_and_stats_2.centroid.x.mean()
/var/folders/9n/1h5nn1bn3ns9xx1s5nvxy5n40000gn/T/ipykernel_61447/136192827.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  y_map = map_and_stats_2.centroid.y.mean()


In [16]:
mymap.choropleth(
geo_data=map_and_stats_2,
name='Choropleth',
data=map_and_stats_2,
columns=['community','count'],
key_on="feature.properties.community",
fill_color='Reds', #https://matplotlib.org/3.5.1/tutorials/colors/colormaps.html          sequential color maps
threshold_scale=myscale,
fill_opacity=0.5,
line_opacity=0.2,
legend_name='Crime Count',
smooth_factor=0
)

/opt/anaconda3/envs/geo_env/lib/python3.10/site-packages/folium/folium.py:409: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


In [17]:
mymap

In [18]:
style_function = lambda x: {'fillColor': '#ffffff',
                            'color':'#000000',
                            'fillOpacity': 0.1,
                            'weight': 0.1}

highlight_function = lambda x: {'fillColor': '#000000',
                                'color':'#000000',
                                'fillOpacity': 0.50,
                                'weight': 0.1}

In [19]:
CrimeCount = folium.features.GeoJson(
    map_and_stats_2,
    style_function=style_function,
    control=False,
    highlight_function=highlight_function,
    tooltip=folium.features.GeoJsonTooltip(
        fields=['community','count'],
        aliases=['Neighborhood: ','Count of Crimes '],
        style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;")
    )
)
mymap.add_child(CrimeCount)
mymap.keep_in_front(CrimeCount)
folium.LayerControl().add_to(mymap)

In [20]:
#this map should have the interactivity now
mymap

In [23]:
df_airbnb = pd.read_csv('./Data/SF_listings.csv')


airbnb_locations = df_airbnb[['latitude','longitude']]
airbnb_list = airbnb_locations.values.tolist()

In [24]:
df_airbnb.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,958,"Bright, Modern Garden Unit - 1BR/1BTH",1169,Holly,NaN,Western Addition,37.77028,-122.43317,Entire home/apt,161,2,314,2021-11-19,3.60,1,126,43,City Registration Pending
1,5858,Creative Sanctuary,8904,Philip And Tania,NaN,Bernal Heights,37.74474,-122.42089,Entire home/apt,235,30,111,2015-08-28,0.76,1,365,0,NaN
2,7918,A Friendly Room - UCSF/USF - San Francisco,21994,Aaron,NaN,Haight Ashbury,37.76555,-122.45213,Private room,56,32,19,2020-03-06,0.17,9,365,0,NaN
3,8142,Friendly Room Apt. Style -UCSF/USF - San Franc...,21994,Aaron,NaN,Haight Ashbury,37.76555,-122.45213,Private room,56,32,8,2017-08-12,0.10,9,365,0,NaN
4,8339,Historic Alamo Square Victorian,24215,Rosy,NaN,Western Addition,37.77564,-122.43642,Entire home/apt,895,7,28,2019-06-28,0.19,2,364,0,STR-0000264


In [25]:
def popup_html(row_num, df_airbnb):
    i = row_num
    name=df_airbnb['name'].iloc[i]
    price=df_airbnb['price'].iloc[i]
    roomtype=df_airbnb['room_type'].iloc[i]


    left_col_color = "#19a7bd"
    right_col_color = "#f2f0d3"

    html = """<!DOCTYPE html>
<html>

<head>
<h4 style="margin-bottom:10"; width="200px">{}</h4>""".format(name) + """

</head>
    <table style="height: 20px; width: 200px; border:1px solid black">
<tbody>
<tr style = "border:1px solid black">
<td style = "border:1px solid black">Price Per Night:</td>
<td style = "border:1px solid black">&nbsp; {}</td>""" .format(price) + """
</tr>

<tr style = "border:1px solid black">
<td style = "border:1px solid black">Room Type:</td>
<td style = "border:1px solid black">&nbsp; {}</td>""" .format(roomtype) + """
</tr>

</tbody>
</table>
</html>
"""
    return html

In [26]:
# set price to 200 for testing
price = 200
df_airbnb_price_filtered = df_airbnb[df_airbnb.price==price]
rows = len(df_airbnb_price_filtered)
#https://georgetsilva.github.io/posts/mapping-points-with-folium/
#https://towardsdatascience.com/folium-map-how-to-create-a-table-style-pop-up-with-html-code-76903706b88a
for point in range(0,rows):
    i = point
    html = popup_html(i,df_airbnb_price_filtered)
    iframe = branca.element.IFrame(html=html,width=200,height=200)
    popup = folium.Popup(folium.Html(html, script=True), max_width=500)
    folium.Marker(airbnb_list[point], popup).add_to(mymap)

In [27]:
mymap

In [19]:
# if you want to save html locally
# mymap.save("index_Chicago.html") 